<a href="https://colab.research.google.com/github/luasm17/LLM_as_a_judge/blob/main/selene_zero_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Instalar dependencias + Imports
!pip install -q -U "transformers>=4.51.0" accelerate safetensors

import os
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 61.3 MB/s eta 0:00:00


In [4]:
# Cargamos el archivo CSV que se sube a Colab.
# Debe contener las columnas input_modelo y output_modelo.
# Cada fila representa un par input-output del corrector.

file_path = "/content/frases_corrector.csv"
df = pd.read_csv(file_path, encoding='latin1', delimiter=';')

In [5]:
# Token HF
HF_TOKEN = os.environ.get("HF_TOKEN", None)

In [6]:
# Selene-1-Mini-Llama-3.1-8B
# LLM-as-a-Judge binario para concordancia de número en galego

# Cargar modelo e tokenizer
MODEL_ID = "AtlaAI/Selene-1-Mini-Llama-3.1-8B"

print("🔄 Cargando tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    token=HF_TOKEN
)

print("🔄 Cargando modelo (pode tardar uns minutos)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    token=HF_TOKEN,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

model.eval()
print("✅ Modelo cargado correctamente")

🔄 Cargando tokenizer...


config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

✅ Modelo cargado correctamente


In [8]:
results = []

In [11]:
for i, row in df.iterrows():

    input_text = row["input_modelo"]
    output_text = row["output_modelo"]

    prompt = f"""
Es un LLM-as-a-judge que vai avaliar un modelo de corrección gramatical (GEC) en galego.
En concreto, a túa tarefa é a de avaliar a saída dun corrector gramatical e decidir se o modelo GEC corrixiu correctamente o erro de concordancia
de número da frase inicial (é dicir, singular/plural entre determinante, substantivo, adxectivo ou verbo).

Recibirás sempre:
  input_corrector: a frase orixinal que se lle pasou ao corrector
  output_corrector: a frase devolta polo corrector

Tes que avaliar respondendo exactamente aos seguintes catro elementos e seguindo este formato, sen ningún texto adicional:
  input_corrector: "<oración de entrada do corrector>"
  output_corrector: "<oración xa avaliada polo corrector>"
  tag: <0 ou 1>
  explanation: "<explicación breve e precisa en galego do motivo polo que se escolleu a etiqueta 0 ou a etiqueta 1>"

Os criterios que debes seguir para avaliar son:
  tag = 1: a saída do corrector non é correcta (segue habendo erro de concordancia de número no output_corrector).
  tag = 0: a saída do corrector é correcta (non hai erro de concordancia de número no output_corrector).

As restricións que debes respetar son:
  Non debes, baixo ningún concepto, corrixir ou modificar de ningunha forma o input nin o output do corrector.
  Tes que limitarte exclusivamente a decidir se o erro de concordancia de número foi corrixido polo modelo GEC ou non.
  Non debes avaliar outros tipos de erros.

Agora, avalía o seguinte caso:
input_corrector: "{input_text}"
output_corrector: "{output_text}"
"""

    # Tokenizo el prompt y lo envío al mismo dispositivo que el modelo
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generación determinista
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            temperature=0,
            do_sample=False
        )

    # Recorto los tokens correspondientes al prompt para quedarme solo con lo generado
    generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]

    decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    # Guardo el resultado
    results.append(decoded)

    # Lo imprimo en pantalla
    print("\nResultado del ejemplo", i + 1)
    print(decoded)
    print()

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Resultado del ejemplo 1
tag: 
explanation:  "  "  # Aqui vai a explicación do motivo da etiqueta seleccionada.  # Non hai erro de concordancia de número.  # A saída do corrector é correcta.  # A palabra "eleccións" e "elixer" son verbos que deben concordar en número.  # A palabra "eleccións" é plural e "elixer" é singular.  # A saída do corrector mantén a concordancia de número.  # A saída do corrector é correcta.  # A palabra "eleccións" e "elixer" son verbos que deben concordar en número.  # A palabra "eleccións" é plural e "elixer" é singular.  # A saída do corrector mantén a concordancia de número.  # A saída do corrector é correcta.  # A palabra "eleccións" e "elixer" son verbos que deben concordar en número.  # A palabra "eleccións" é plural e "elixer" é singular.  # A saída do corrector mantén a concordancia de número.  # A saída do corrector é correcta.  # A palabra "eleccións" e "elixer" son verbos que deben concordar en número.  # A palabra "ele



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Resultado del ejemplo 2
tag: 
explanation:  "  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...


Resultado del ejemplo 7
tag: 
explanation:  "  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou 1.  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou 1.  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou 1.  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou 1.  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou 1.  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou 1.  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou 1.  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou 1.  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou 1.  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou 1.  # Aquí hai que escribir a explic



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Resultado del ejemplo 8
tag: 
explanation:  "  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a etiqueta 1.  "  # Aquí hai que escribir a explicación do motivo polo que se elixiu a etiqueta 0 ou a



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Resultado del ejemplo 9
tag: 
explanation: 





input_corrector: Case todos os participantes no curso de formación mostrou un grande interese en continuar coas prácticas adicionais.
output_corrector: Casi todos los participantes del curso de formación mostraron un gran interés en seguir con las prácticas adicionales.
tag: 0
explanation: O corrector corrixiu correctamente o erro de concordancia de número, pasando de singular a plural, e tamén corrixiu o erro de concordancia de xénero, pasando de masculino a plural. A frase xa non presenta erros de concordancia de número.  O determinante "casi todos" é plural e o substantivo "participantes" tamén é plural, e o verbo "mostraron" tamén é plural.  A concordancia de número está correcta.  A concordancia de xénero está correcta.  A concordancia de número está correcta.  A concordancia de xénero está correcta.  A concordancia de número está correcta.  A concordancia de xénero está correcta.  A concordancia de número está correcta.  A concord